---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [ ]:
import pandas as pd
import numpy as np
import re
import datetime as dt
from datetime import datetime
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [ ]:
def get_list_of_university_towns():

    univ_towns = open('university_towns.txt', encoding='utf-8').readlines()

    state_list = []
    town_list = []

    for name in univ_towns:
        if '[ed' in name:
            statename = re.sub('\[edit]\n$', '', name)
        elif '(' in name:
            townname = name.rstrip()
            townname = townname.split('(')[0].strip()
            town_list.append(townname)
            state_list.append(statename)
        elif ':' in name:
            townname = name.rstrip()
            town_list.append(townname)
            state_list.append(statename)
        else:
            townname = name.rstrip()
            town_list.append(townname)
            state_list.append(statename)

    townsdf = pd.DataFrame()
    townsdf['State'] = state_list
    townsdf['RegionName'] = town_list
    
    return townsdf

get_list_of_university_towns()

In [ ]:
def get_recession_start():
    gdp = pd.read_excel('gdplev.xls', skiprows = 219, usecols = [4, 6], names=['Quarter', 'GDP in Billions'])
    
    gdp['GDP Change'] = gdp['GDP in Billions'].diff()

    for i, row in gdp.iterrows():
        if (gdp['GDP Change'][i] < 0) & (gdp['GDP Change'][i+1] < 0):
            recession_start = gdp['Quarter'][i]
            break

    return recession_start 

get_recession_start()

In [ ]:
def get_recession_end():
    
    gdp = pd.read_excel('gdplev.xls', skiprows = 219, usecols = [4, 6], names=['Quarter', 'GDP in Billions'])
    
    gdp['GDP Change'] = gdp['GDP in Billions'].diff()
    
    recession_start = get_recession_start()
    recession_start_data = gdp.loc[gdp['Quarter'] == recession_start]
    recession_start_index = recession_start_data.index[0]
    
    recessiondf = gdp[recession_start_index:recession_start_index+10]

    for i, row in recessiondf.iterrows():
        if (gdp['GDP Change'][i] > 0) & (gdp['GDP Change'][i+1] > 1):
            recession_end = gdp['Quarter'][i+1]
            break

    return recession_end

get_recession_end()

In [ ]:
def get_recession_bottom():
    
    gdp = pd.read_excel('gdplev.xls', skiprows = 219, usecols = [4, 6], names=['Quarter', 'GDP in Billions'])

    gdp['GDP Change'] = gdp['GDP in Billions'].diff()

    recession_start = get_recession_start()
    recession_start_data = gdp.loc[gdp['Quarter'] == recession_start]
    recession_start_index = recession_start_data.index[0]

    recession_end = get_recession_end()
    recession_end_data = gdp.loc[gdp['Quarter'] == recession_end]
    recession_end_index = recession_end_data.index[0]

    recessiondf = gdp[recession_start_index:recession_end_index+1]

    recession_bottom = recessiondf[recessiondf['GDP in Billions']==recessiondf['GDP in Billions'].min()]

    return recession_bottom.iloc[0,0]

get_recession_bottom()

In [ ]:
def convert_housing_data_to_quarters():
    
    housing = pd.read_csv('City_Zhvi_AllHomes.csv')

    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    housing['State'] = housing['State'].map(states)
    housing = housing.set_index(['State', 'RegionName'])

    #housing = housing.groupby(level=[0,1]).count()

    housing = housing.iloc[:,49:]      

    #housing.iloc[:-1] = housing.iloc[:-1].apply(pd.to_datetime, errors='coerce')

    #housing = housing.groupby(pd.PeriodIndex(housing.columns, freq='Q'), axis=1).mean()

    housing.columns = pd.to_datetime(housing.columns, format='%Y-%m-%d', errors='coerce')

    housing = housing.resample('Q',axis=1).mean()

    housing = housing.rename(columns=lambda x: str(x.to_period('Q')).lower())

    return housing

convert_housing_data_to_quarters()

In [ ]:
def run_ttest():

    housingdf = convert_housing_data_to_quarters()
    rec_start = get_recession_start()
    rec_bottom = get_recession_bottom()
    univdf = get_list_of_university_towns()

    gdp = pd.read_excel('gdplev.xls', skiprows = 219, usecols = [4, 6], names=['Quarter', 'GDP in Billions'])
    gdp['GDP Change'] = gdp['GDP in Billions'].diff()

    rec_start_index = gdp.loc[gdp['Quarter'] == rec_start].index[0]
    index_before_rec_start = rec_start_index-1
    q_bfr_rec_start = gdp['Quarter'][index_before_rec_start]

    housingdf = housingdf[[q_bfr_rec_start, rec_bottom]]
    housingdf['PriceRatio'] = ((housingdf[q_bfr_rec_start])-(housingdf[rec_bottom]))/(housingdf[q_bfr_rec_start])
    housingdf = housingdf.dropna()
    housingdf = housingdf.reset_index()

    univ_housing = pd.merge(housingdf, univdf, how='inner', on=['State', 'RegionName'])
    non_univ_housing = housingdf[~housingdf.index.isin(univ_housing.index)]

    t_stat, p_value = ttest_ind(non_univ_housing['PriceRatio'], univ_housing['PriceRatio'], nan_policy='omit')
    if p_value < 0.01:
        different = True
    else:
        different = False
    if (univ_housing['PriceRatio'].mean()) < (non_univ_housing['PriceRatio'].mean()): 
        better = "university town" 
    else:
        better = "non-university town"
    
    return (different, p_value, better)

run_ttest()